In [35]:
import pandas as pd
import selenium
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import multiprocessing
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from multiprocessing import Pool
import numpy as np
import time

from selenium.common.exceptions import TimeoutException

In [2]:
import sys
from pathlib import Path
sys.path.append(str(Path("../lib/").absolute()))

In [31]:
def run_scraper(coords):
    profile = FirefoxProfile()
    profile.set_preference("javascript.enabled", True)
    driver = webdriver.Firefox(profile)
    driver.implicitly_wait(10)
    
    for lat, lon, zoom in coords:
        driver.get("https://www.openstreetmap.org/#map=%d/%f/%f" % (zoom, lat, lon))
        time.sleep(5)
        driver.save_screenshot("screenshots/%d-%f-%f.png" % (zoom, lat, lon))
    
    driver.quit()
    

In [32]:
def scrape_pool(all_coords, num_processes=2):
    pool = Pool(num_processes)
    manager = multiprocessing.Manager()
    coords = np.array_split(all_coords, num_processes)

    for i in range(num_processes):
        pool.apply_async(run_scraper, args=(coords[i], ))

    pool.close()
    pool.join()

In [15]:
all_coords = [(33.7401, -84.3158, 18)]
scrape_pool(all_coords)

In [17]:
orig_train = pd.read_csv('train.csv')
orig_test = pd.read_csv('test.csv')

In [25]:
full_df = pd.concat([orig_train[['Latitude', 'Longitude']], orig_test[['Latitude', 'Longitude']]])
full_df['zoom'] = 16

In [ ]:
scrape_pool(full_df.drop_duplicates().values, num_processes=4)